In [1]:
#聲音處理套件
import librosa
import librosa.display
#資料處理套件
import pandas as pd
import numpy as np
import torch

In [2]:
#音訊資訊CSV檔位置
CSV_FILE_PATH = "D:/dataset/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/dataset/ESC-50-master/ESC-50-master/audio/"

In [3]:
#讀取CSV檔
df = pd.read_csv(CSV_FILE_PATH)
df.head()

#篩選出ESC10作為訓練資料
df_10 = df[df['esc10']==True]
#刪去無用欄位
df_10 = df_10.drop(['esc10','src_file','take'], axis=1)

#取出類別，去除重複並排序
classes = df_10['category'].unique()
class_dict = {i:x for x,i in enumerate(classes)}
print(class_dict)

#重新標label
df_10['target'] = df_10['category'].map(class_dict)

{'dog': 0, 'chainsaw': 1, 'crackling_fire': 2, 'helicopter': 3, 'rain': 4, 'crying_baby': 5, 'clock_tick': 6, 'sneezing': 7, 'rooster': 8, 'sea_waves': 9}


In [4]:
class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

In [5]:
#ESC資料集聲音處理
def trainPreprocess(fold, soundLength):
    #聲音長度
    soundLength = soundLength
    #一個聲音切成幾段
    soundCount = 6 - soundLength
    
    #要回傳的資料 
    #資料 標籤 是否為電鋸聲/正常ESC10標籤
    x_train, y_train_2, y_train_10 = [], [], []
    x_val, y_val_2, y_val_10 = [], [], []
    
    #根據fold來切分train/val
    train_df = df_10[df_10.fold != fold]
    val_df = df_10[df_10.fold == fold]
    
    #轉換train資料/添加標籤
    for data in train_df.itertuples():
        sig , sr = librosa.load(DATA_PATH+data[1], sr=conf.sr)
        for i in range(soundCount):
            mel_spec = sig[i*sr : int((i+soundLength)*sr)]
            x_train.append(mel_spec)
            #正常標籤
            y_train_10.append(data[3])
            #在ESC-10中 電鋸聲標籤為1
            if(data[3]==1):
                y_train_2.append(1)
            else:
                y_train_2.append(0)
            
    #轉換val資料/添加標籤
    for data in val_df.itertuples():
        sig , sr = librosa.load(DATA_PATH+data[1], sr=conf.sr)
        for i in range(soundCount):
            mel_spec = sig[i*sr : int((i+soundLength)*sr)]
            x_val.append(mel_spec)
            #正常標籤
            y_val_10.append(data[3])
            #在ESC-10中 電鋸聲標籤為1
            if(data[3]==1):
                y_val_2.append(1)
            else:
                y_val_2.append(0)

          
    #通通轉成 torch 的 tensor 形式
    x_train = torch.Tensor(x_train)
    y_train_10 = torch.Tensor(y_train_10).type(torch.LongTensor)
    y_train_2 = torch.Tensor(y_train_2).type(torch.LongTensor)
    x_val = torch.Tensor(x_val) 
    y_val_10 = torch.Tensor(y_val_10).type(torch.LongTensor)
    y_val_2 = torch.Tensor(y_val_2).type(torch.LongTensor)
    
    #轉維度 BCW
    x_train = torch.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
    x_val = torch.reshape(x_val, (x_val.shape[0], 1, x_val.shape[1]))
    
    #存檔位置
    ESC_DATA_PATH = "D:/paper/code/Preprocessing/ESC_1D/Fold"
    x_train_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/x_train.pt"
    y_train_10_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/y_train_10.pt" 
    y_train_2_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/y_train_2.pt" 
    x_val_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/x_val.pt" 
    y_val_10_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/y_val_10.pt" 
    y_val_2_path = ESC_DATA_PATH + str(fold) + "/" + str(soundLength) + "s/y_val_2.pt" 
    
    torch.save(x_train, x_train_path)
    torch.save(y_train_10, y_train_10_path)
    torch.save(y_train_2, y_train_2_path)
    torch.save(x_val, x_val_path)
    torch.save(y_val_10, y_val_10_path)
    torch.save(y_val_2, y_val_2_path)

In [6]:
for i in range(1,6):
    for s in range(1,6):
        trainPreprocess(i,s)

C:\Users\hu\AppData\Local\Temp\ipykernel_10756\1658287104.py:47: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  x_train = torch.Tensor(x_train)
